## step 1: imports

In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
pd.set_option('display.max_columns', None)

In [22]:
companies = pd.read_csv('Modified_Unicorn_Companies.csv')
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG"
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita..."


## step 2: data cleaning 

In [23]:
companies.dtypes

Company             object
Valuation            int64
Date Joined         object
Industry            object
City                object
Country/Region      object
Continent           object
Year Founded         int64
Funding             object
Select Investors    object
dtype: object

In [24]:
# converting the 'object' dtype column to 'datetime'
companies["Date Joined"] = pd.to_datetime(companies["Date Joined"])

In [25]:
# adding a column called 'Years To Unicorn'
companies["Years To Unicorn"] = companies["Date Joined"].dt.year - companies["Year Founded"]

In [26]:
companies.head()

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
0,Bytedance,180,2017-04-07,Artificial intelligence,Beijing,China,Asia,2012,$8B,"Sequoia Capital China, SIG Asia Investments, S...",5
1,SpaceX,100,2012-12-01,Other,Hawthorne,United States,North America,2002,$7B,"Founders Fund, Draper Fisher Jurvetson, Rothen...",10
2,SHEIN,100,2018-07-03,E-commerce & direct-to-consumer,Shenzhen,China,Asia,2008,$2B,"Tiger Global Management, Sequoia Capital China...",10
3,Stripe,95,2014-01-23,FinTech,San Francisco,United States,North America,2010,$2B,"Khosla Ventures, LowercaseCapital, capitalG",4
4,Klarna,46,2011-12-12,Fintech,Stockholm,Sweden,Europe,2005,$4B,"Institutional Venture Partners, Sequoia Capita...",6


#### input validation

##### correcting bad data

In [27]:
companies["Years To Unicorn"].describe()

count    1074.000000
mean        7.013035
std         5.331842
min        -3.000000
25%         4.000000
50%         6.000000
75%         9.000000
max        98.000000
Name: Years To Unicorn, dtype: float64

'Years To Unicorn' cannot be negative.

In [28]:
companies[companies['Years To Unicorn'] < 0]

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2020,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


In [31]:
print(companies.loc[companies['Company']=='InVision', 'Year Founded'])
companies.loc[companies['Company']=='InVision', 'Year Founded'] = 2011
# an internet search reveals that InVision was founded in 2011

527    2020
Name: Year Founded, dtype: int64


In [32]:
companies[companies['Company']=='InVision']
# checking out again

,Company,Valuation,Date Joined,Industry,City,Country/Region,Continent,Year Founded,Funding,Select Investors,Years To Unicorn
527,InVision,2,2017-11-01,Internet software & services,New York,United States,North America,2011,$349M,"FirstMark Capital, Tiger Global Management, IC...",-3


In [35]:
# recalculating all the values for 'Years To Unicorn'
companies["Years To Unicorn"] = companies["Date Joined"].dt.year - companies["Year Founded"]

# checking the final result
companies.loc[companies['Company']=='InVision', 'Years To Unicorn']

527    6
Name: Years To Unicorn, dtype: int64

In [37]:
# a list of all the industry name that should be in the dataset. any labels in the `Industry` column that are not in `industry_list` are misspellings.

industry_list = ['Artificial intelligence', 'Other','E-commerce & direct-to-consumer', 'Fintech',
       'Internet software & services','Supply chain, logistics, & delivery', 'Consumer & retail',
       'Data management & analytics', 'Edtech', 'Health', 'Hardware','Auto & transportation',
        'Travel', 'Cybersecurity','Mobile & telecommunications']

In [ ]:
# checking which values are in 'Industry' but not in industry_list.

In [39]:
# first way
set(companies['Industry']) - set(industry_list)

{'Artificial Intelligence', 'Data management and analytics', 'FinTech'}

In [40]:
# second way
mask = ~companies['Industry'].isin(industry_list)
companies[mask]["Industry"].unique()

array(['FinTech', 'Data management and analytics',
       'Artificial Intelligence'], dtype=object)

In [51]:
## correcting the bad entries with 'replace()' method.

# companies["Industry"] = companies["Industry"].replace("Artificial Intelligence", "Artificial intelligence")
#
# companies["Industry"] = companies["Industry"].replace("Data management and analytics", "Data management & analytics")
#
# companies["Industry"] = companies["Industry"].replace("FinTech", "Fintech")


## a way simpler and way shorter way to do this operation:

replacement_dict = {
    "Artificial Intelligence" : "Artificial intelligence",
    "Data management and analytics" : "Data management & analytics",
    "FinTech" : "Fintech",
}

companies["Industry"] = companies["Industry"].replace(replacement_dict)

In [52]:
# checking if there are andy misspelling values still
set(companies['Industry']) - set(industry_list)

set()